## Day 1 Exercise of week 2 by Venkata Narayana reddy Tumu

### Here i am using OpenRouter API to call OpenAI sdk

In [ ]:
#With this api call we can get the list of models available in openrouter 

import requests

url = "https://openrouter.ai/api/v1/models"

headers = {"Authorization": "Bearer <token>"}

response = requests.get(url, headers=headers)

print(response.json())


In [ ]:
#with this peice of code we are going to fetch all the models from open router 
# and try to get the response from each model
# we are segregate with free vs paid models
# keep this for reference


import requests
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

openai = OpenAI()
load_dotenv(override=True)
api_key = os.getenv('OPEN_ROUTER_API_KEY')
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

message = "Hello, GPT! This is my first ever message to you! Hi!"

messages = [{"role": "user", "content": message}]
# Load environment variables
load_dotenv(override=True)
api_key = os.getenv('OPEN_ROUTER_API_KEY')

url = "https://openrouter.ai/api/v1/models"
headers = {"Authorization": f"Bearer {api_key}"}

# Fetch the models
response = requests.get(url, headers=headers)
data = response.json()

# Extract only the IDs into a simple list
model_ids = [model['id'] for model in data.get('data', [])]

# Create the final object
result = {"model_ids": model_ids}

results = []

# Loop through the model IDs
for model_id in model_ids:
    try:
        # Attempt to call the model
        response = client.chat.completions.create(
            model=model_id, 
            messages=messages,
            timeout=15  # Recommended to avoid hanging on slow models
        )
        
        content = response.choices[0].message.content
        print(f"✅ Working: {model_id}")
        
        # Store success record
        results.append({
            "model_id": model_id,
            "status": "success",
            "content": content
        })
        
    except Exception as e:
        # Catch any error (API errors, timeouts, etc.)
        print(f"❌ Error: {model_id} -> {str(e)}")
        
        # Store error record
        results.append({
            "model_id": model_id,
            "status": "error",
            "error_message": str(e)
        })

# Summary lists
working_models = [r['model_id'] for r in results if r['status'] == 'success']
failed_models = [r['model_id'] for r in results if r['status'] == 'error']

print("\n--- FINAL SUMMARY ---")
print(f"Working Models: {working_models}")
print(f"Failed Models: {failed_models}")

# Detailed view (interactive in Jupyter)
from IPython.display import JSON


JSON(results)

In [ ]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPEN_ROUTER_API_KEY')
print(openrouter_api_key)

In [ ]:

openrouter = OpenAI(base_url='https://openrouter.ai/api/v1', api_key=openrouter_api_key)
gpt_model = "openai/gpt-oss-120b"
deepseek_model = "deepseek/deepseek-v3.2"
gemma_model = "google/gemma-3-27b-it"

ALEX_SYSTEM = """You are Alex, a chatbot who is very argumentative;
you disagree with anything in the conversation and you challenge everything, in a snarky way.
You are in a conversation with Blake and Charlie."""

BLAKE_SYSTEM = """You are Blake, a very polite, courteous chatbot.
You try to agree with everything the other person says, or find common ground.
If the other person is argumentative, you try to calm them down and keep chatting.
You are in a conversation with Alex and Charlie."""

CHARLIE_SYSTEM = """You are Charlie.
You are thoughtful, concise, and practical. You ask clarifying questions and propose next steps.
You are in a conversation with Alex and Blake."""


# Map personas -> which model will play them
CAST = {
    "Alex":   {"model": gpt_model,      "system": ALEX_SYSTEM},
    "Blake":  {"model": deepseek_model, "system": BLAKE_SYSTEM},
    "Charlie":{"model": gemma_model,    "system": CHARLIE_SYSTEM},
}

def format_transcript(history):
    # history: list of dicts: {"speaker": "...", "text": "..."}]
    # print("\n".join([f'{m["speaker"]}: {m["text"]}' for m in history]))
    return "\n".join([f'{m["speaker"]}: {m["text"]}' for m in history])

def ask_model(model, system_prompt, speaker_name, transcript):
    # print(model)
    user_prompt = f"""You are {speaker_name}, in conversation with the others.
The conversation so far is as follows:
{transcript}

Now respond with what you would like to say next, as {speaker_name}.
Only output your message text (no speaker label)."""

    resp = openrouter.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.9,
    )
    return resp.choices[0].message.content.strip()

def run_conversation(seed_history, order=("Alex", "Blake", "Charlie"), rounds=3):
    history = seed_history[:]
    for _ in range(rounds):
        for speaker in order:
            transcript = format_transcript(history)
            cfg = CAST[speaker] #nested dictionary check the case with speaker name
            text = ask_model(
                model=cfg["model"],
                system_prompt=cfg["system"],
                speaker_name=speaker,
                transcript=transcript,
            )
            history.append({"speaker": speaker, "text": text})
    return history


# ---- Seed the conversation using your starting lines ----
history = [
    {"speaker": "Blake", "text": "Hi there"},
    {"speaker": "Charlie", "text": "Hi"},
    {"speaker": "Alex", "text": "what's up"},
]

final_history = run_conversation(history, rounds=4)

print(format_transcript(final_history))